# Detox - Chat Message Toxicity Detector
## ADI1302-SPARK SCALA FUNDAMENTALS

**Student Name:** SARAVANA PRIYAN S T  
**Registration Number:** 927623BAD100

---

This notebook provides an interactive environment for training and analyzing the toxicity detection model using PySpark.

### Project Overview
- **Objective:** Detect toxic messages in chat data using PySpark and Machine Learning
- **Tech Stack:** PySpark, Spark MLlib, Logistic Regression
- **Dataset:** Jigsaw Toxic Comments Classification Dataset

In [ ]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, when, length
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

## 1. Initialize Spark Session

Configure and create a Spark session with optimized settings for toxicity detection.

In [ ]:
# Create Spark Session with Web UI enabled
spark = SparkSession.builder \
    .appName("Detox-Toxicity-Detector-Notebook") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.ui.port", "4040") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark Session Created")
print(f"  - Spark Version: {spark.version}")
print(f"  - Application ID: {spark.sparkContext.applicationId}")
print(f"  - Spark Web UI: http://localhost:4040")

## 2. Load and Explore Data

Load the toxic comments dataset and perform initial exploration.

In [ ]:
# Load dataset
data_path = "data/chat_data.csv"

df = spark.read.csv(
    data_path,
    header=True,
    inferSchema=True
)

# Display schema
print("Dataset Schema:")
df.printSchema()

print(f"\nTotal Records: {df.count()}")

In [ ]:
# Show sample data
print("Sample Data:")
df.show(5, truncate=50)

In [ ]:
# Basic statistics
print("Column Names:")
print(df.columns)

# Check for null values
print("\nNull Value Counts:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

## 3. Data Preprocessing

Clean and prepare the text data for modeling.

In [ ]:
# Import preprocessing modules
import sys
sys.path.append('/home/saravana/projects/ssfproject')

from preprocessing import TextPreprocessor
from data_ingestion import DataIngestion

# Initialize components
data_ingestion = DataIngestion(spark)
preprocessor = TextPreprocessor()

# Validate data
df_clean = data_ingestion.validate_data(df)

# Get statistics
stats = data_ingestion.get_data_statistics(df_clean)

print(f"✓ Data cleaned and validated")
print(f"  - Clean records: {df_clean.count()}")

In [ ]:
# Preprocess text
df_processed = preprocessor.preprocess_text(df_clean)
df_processed = preprocessor.create_label_column(df_processed, label_col="toxic")

# Show sample processed data
print("Processed Data Sample:")
df_processed.select("comment_text", "cleaned_text", "label").show(5, truncate=50)

## 4. Feature Engineering

Extract features using TF-IDF vectorization.

In [ ]:
# Build and fit feature pipeline
df_features = preprocessor.fit_transform_features(df_processed)

print("✓ Features extracted successfully")
print("\nFeature columns:")
print(df_features.columns)

# Cache for performance
df_features.cache()
print(f"\n✓ Dataset cached ({df_features.count()} records)")

## 5. Train-Test Split

Split the data into training and testing sets.

In [ ]:
# Split data (80% train, 20% test)
train_df, test_df = df_features.randomSplit([0.8, 0.2], seed=42)

train_count = train_df.count()
test_count = test_df.count()

print(f"✓ Data split completed")
print(f"  - Training set: {train_count} records ({train_count/(train_count+test_count)*100:.1f}%)")
print(f"  - Test set: {test_count} records ({test_count/(train_count+test_count)*100:.1f}%)")

# Cache splits
train_df.cache()
test_df.cache()

## 6. Model Training

Train a Logistic Regression model for toxicity classification.

In [ ]:
from model import ToxicityClassifier

# Initialize classifier
classifier = ToxicityClassifier()

# Train model
print("Training Logistic Regression model...")
print("This may take a few minutes...\n")

model = classifier.train_model(train_df)

print("\n✓ Model training completed")

## 7. Model Evaluation

Evaluate the model performance on the test set.

In [ ]:
# Evaluate model
metrics = classifier.evaluate_model(test_df)

# Visualize metrics
metrics_df = pd.DataFrame({
    'Metric': ['AUC-ROC', 'AUC-PR', 'Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Score': [
        metrics['auc'],
        metrics['auc_pr'],
        metrics['accuracy'],
        metrics['precision'],
        metrics['recall'],
        metrics['f1_score']
    ]
})

plt.figure(figsize=(10, 6))
sns.barplot(data=metrics_df, x='Metric', y='Score', palette='viridis')
plt.title('Model Performance Metrics', fontsize=16, fontweight='bold')
plt.ylabel('Score', fontsize=12)
plt.xlabel('Metric', fontsize=12)
plt.ylim(0, 1)
plt.xticks(rotation=45)

# Add value labels on bars
for i, v in enumerate(metrics_df['Score']):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Model evaluation completed")

## 8. Make Predictions

Generate predictions for all messages in the dataset.

In [ ]:
# Make predictions
predictions_df = classifier.predict(df_features)

print("✓ Predictions generated")
print("\nPrediction Sample:")
predictions_df.select(
    "id", 
    "comment_text", 
    "toxicity_score", 
    "toxicity_level", 
    "prediction"
).show(10, truncate=50)

# Cache predictions
predictions_df.cache()

In [ ]:
# Analyze prediction distribution
print("Toxicity Level Distribution:")
predictions_df.groupBy("toxicity_level").count().orderBy("count", ascending=False).show()

print("\nPrediction Distribution:")
predictions_df.groupBy("prediction").count().show()

## 9. User-Level Analysis

Aggregate toxicity metrics at the user level.

In [ ]:
from user_analysis import UserToxicityAnalyzer

# Initialize analyzer
analyzer = UserToxicityAnalyzer()

# Aggregate user toxicity
user_aggregates = analyzer.aggregate_user_toxicity(predictions_df)

print("✓ User-level aggregation completed")
print("\nTop 10 Most Toxic Users:")
user_aggregates.select(
    "user_id",
    "total_messages",
    "avg_toxicity_score",
    "max_toxicity_score",
    "user_toxicity_level",
    "toxic_messages_count"
).show(10, truncate=False)

In [ ]:
# Get user statistics
user_stats = analyzer.get_user_statistics(user_aggregates)

## 10. Visualizations

Create comprehensive visualizations of toxicity patterns.

In [ ]:
# Convert to Pandas for visualization
user_agg_pd = user_aggregates.select(
    "user_id",
    "total_messages",
    "avg_toxicity_score",
    "user_toxicity_level",
    "toxic_messages_count"
).toPandas()

print(f"✓ Converted {len(user_agg_pd)} user records to Pandas")

In [ ]:
# Visualization 1: User Toxicity Level Distribution
plt.figure(figsize=(12, 6))

level_order = ['MINIMAL', 'LOW', 'MODERATE', 'HIGH', 'VERY_HIGH']
level_counts = user_agg_pd['user_toxicity_level'].value_counts()

plt.subplot(1, 2, 1)
sns.countplot(data=user_agg_pd, y='user_toxicity_level', order=level_order, palette='RdYlGn_r')
plt.title('User Distribution by Toxicity Level', fontsize=14, fontweight='bold')
plt.xlabel('Number of Users', fontsize=11)
plt.ylabel('Toxicity Level', fontsize=11)

# Visualization 2: Average Toxicity Score Distribution
plt.subplot(1, 2, 2)
plt.hist(user_agg_pd['avg_toxicity_score'], bins=50, color='coral', edgecolor='black', alpha=0.7)
plt.axvline(0.5, color='red', linestyle='--', linewidth=2, label='Moderate Threshold')
plt.axvline(0.7, color='darkred', linestyle='--', linewidth=2, label='High Threshold')
plt.title('Distribution of Average Toxicity Scores', fontsize=14, fontweight='bold')
plt.xlabel('Average Toxicity Score', fontsize=11)
plt.ylabel('Number of Users', fontsize=11)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 3: Top 20 Most Toxic Users
top_20_users = user_agg_pd.nlargest(20, 'avg_toxicity_score')

plt.figure(figsize=(14, 8))
sns.barplot(data=top_20_users, y='user_id', x='avg_toxicity_score', palette='Reds_r')
plt.title('Top 20 Most Toxic Users', fontsize=16, fontweight='bold')
plt.xlabel('Average Toxicity Score', fontsize=12)
plt.ylabel('User ID', fontsize=12)
plt.xlim(0, 1)

# Add score labels
for i, v in enumerate(top_20_users['avg_toxicity_score']):
    plt.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Visualization 4: Messages vs Toxicity Score
plt.figure(figsize=(12, 6))
scatter_sample = user_agg_pd.sample(min(1000, len(user_agg_pd)))

plt.scatter(
    scatter_sample['total_messages'],
    scatter_sample['avg_toxicity_score'],
    c=scatter_sample['avg_toxicity_score'],
    cmap='RdYlGn_r',
    alpha=0.6,
    s=100,
    edgecolors='black',
    linewidth=0.5
)
plt.colorbar(label='Toxicity Score')
plt.title('User Messages vs Average Toxicity Score', fontsize=16, fontweight='bold')
plt.xlabel('Total Messages', fontsize=12)
plt.ylabel('Average Toxicity Score', fontsize=12)
plt.axhline(0.5, color='orange', linestyle='--', alpha=0.7, label='Moderate Threshold')
plt.axhline(0.7, color='red', linestyle='--', alpha=0.7, label='High Threshold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Export Results

Save predictions and user aggregates to CSV files.

In [ ]:
from pyspark.sql.functions import substring

# Export predictions
output_predictions = "output/toxicity_predictions.csv"
predictions_df.select(
    "id",
    substring("id", 1, 8).alias("user_id"),
    "comment_text",
    "toxicity_score",
    "toxicity_level",
    "prediction",
    "toxic",
    "severe_toxic",
    "obscene",
    "threat",
    "insult",
    "identity_hate"
).coalesce(1).write.mode("overwrite").csv(output_predictions, header=True)

print(f"✓ Predictions saved to: {output_predictions}")

# Export user aggregates
output_users = "output/user_toxicity_levels.csv"
user_aggregates.coalesce(1).write.mode("overwrite").csv(output_users, header=True)

print(f"✓ User toxicity levels saved to: {output_users}")

In [ ]:
# Save model
model_path = "models/toxicity_model"
classifier.save_model(model_path)

print(f"✓ Model saved to: {model_path}")

## 12. Summary and Insights

### Key Findings:
1. **Model Performance**: Achieved strong performance with high accuracy and AUC scores
2. **User Behavior**: Identified users with varying levels of toxicity
3. **Toxicity Patterns**: Analyzed distribution of toxic content across the dataset

### Next Steps:
- Fine-tune model parameters for improved performance
- Implement real-time toxicity detection
- Add support for multi-class toxicity classification
- Integrate with chat platforms for live monitoring

---

**Project Completed Successfully! 🎉**

In [ ]:
# Display final statistics
print("="*60)
print("FINAL PROJECT STATISTICS")
print("="*60)
print(f"Total Records Processed: {df_features.count()}")
print(f"Model Accuracy: {metrics['accuracy']:.4f}")
print(f"Total Users Analyzed: {user_stats['total_users']}")
print(f"Overall Toxicity Rate: {user_stats['toxicity_rate']:.2f}%")
print("="*60)
print("\n✓ DETOX PROJECT COMPLETED SUCCESSFULLY!")
print("\nSpark Web UI: http://localhost:4040")

## Cleanup

Stop the Spark session when done.

In [ ]:
# Uncomment to stop Spark session
# spark.stop()
# print("✓ Spark session stopped")